<a href="https://colab.research.google.com/github/andrebelem/pythonverse/blob/main/Ploting_ERA5_data_with_Proplot_using_COLAB_%5BPUBLIC%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](http://observatoriooceanografico.org/wp-content/uploads/elementor/thumbs/logo-o2-2-p31vla9ib17i85olifd4l1xk0e5lktrkal6u0s4c6o.png)<br>
# How to read ERA5 data via API and plot with Proplot
### An example for Earth Sciences and Climate Change students
#### Preferential plataforms: Jupyter Notebook/Jupyter Lab/COLAB.
<br>

Created by [A.L. Belém](mailto:andrebelem@id.uff.br) @ [Oceanographic Observatory](http://www.observatoiriooceanografico.uff.br) 2021 (version 1) for colaborators in the O2 (and for those who are most interested)<br>

History:<br>
[April 12th, 2021 Version 1] Main example<br>
[May 28th, 2021] Adapted to COLAB (ProPlot)<br>



**NOTA: If you modify this notebook, be careful not to distribute your sensitive data to access the CDS. Check before distributing**

This notebook was *stimulated* by participating in the **Introduction to Scientific Python** course given by the meteorologist [Willy Hagi](https://linktr.ee/willyhagi) from [Meteonorte](https://www.meteonorte.com/) between April 10 and 11, 2021, and based on *lessons learned* from the [Luke Gloege](https://lukegloege.medium.com/) in [Towards data science](https://towardsdatascience.com/read-era5-directly-into-memory-with-python-511a2740bba0) about the [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview) data.

The idea here is to mix the ERA5 and the [Proplot](https://proplot.readthedocs.io/en/latest/) (and [Cartopy](https://scitools.org.uk/cartopy/docs/v0.13/index.html)). `CartoPy` is a package created by **UK Met Office** with the aim of allowing cartographic projections and plotting various types of geospatial data in Python. `Proplot` is an excellent library that facilitates the plotting of maps based on `CartoPy`, much more *clean* and *direct* than with the native package. **Note for COLAB**: There are a number of difficulties to run `ProPlot` on COLAB, but with a little patience, you can configure your virtual machine to (re)install the necessary packages. See below throughout the text.

ERA5 is a fifth generation climate reanalysis produced by the European Center for Medium-Term Meteorological Forecasts (ECMWF) and it provides a time output for many atmospheric, land surface and ocean parameters, along with some uncertainty estimates. The volume of data is immense, as well as its potential, which makes routine work with local data almost impossible if you try to download the whole thing. Working in the cloud is the way out to not take up space and consume internet bandwidth (which is quite inconsistent for those concerned with the climate x energy). This notebook was created as a "cake recipe" for the installation of the climatic data storage API (CDS), which will allow the reading and analysis of ERA5 data inside Python.

### The basics of ERA5 and the CDS API

Access via CDS API allows work on data sets from ERA5 and ERA5-land (the same dataset but only on land). There are four main subsets of ERA5: hourly and monthly products, each at pressure levels (including air fields at altitude) and simple levels or single levels (atmospheric, oceanic and terrestrial surface quantities). ERA5-land only includes single levels. If you are in a hurry, then ...
The four ERA5 subsets are (take a tour at [Copernicus data hub](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset)):
- ERA5 hourly data at single levels
- Average monthly data ERA5 at individual levels
- Hourly data ERA5 on pressure levels
- Average monthly data ERA5 on pressure levels

The two ERA5-land subsets:
- ERA5-land hourly data
- ERA5-land monthly data

The ERA5 reanalysis is described in detail in the paper Hersbach, H, Bell, B, Berrisford, P, et al. The ERA5 global reanalysis. Q J R Meteorol Soc. 2020; 146: 1999– 2049. https://doi.org/10.1002/qj.3803. Please, give them the credit. 

#### First, reconfiguring your machine<br>

**These steps below are exclusively for COLAB**<br>

To get you all the packages you need, let's (1) update the machine packages and install what you need, (2) resolve the dispute between `shapely` and `cartopy`, and (3) install `proplot` . <br>
**Attention:** this operation can take 2 to 3 minutes. Get a coffee and have a little patience.

In [ ]:
# ** exclusivo para o COLAB
# essa foi a receita de bolo final que funcionou afinal. Note que ele atualiza os pacotes da maquina primeiro
!grep '^deb ' /etc/apt/sources.list | \
  sed 's/^deb /deb-src /g' | \
  tee /etc/apt/sources.list.d/deb-src.list
!apt-get update

# depois instala o cartopy e remove o shapely (que não conversa com o cartopy no colab)
!apt-get -qq build-dep python3-cartopy
!apt-get -qq remove python-shapely python3-shapely
# e por fim reinstala o shapely e o cartopy na versão necessária
!pip install --no-binary shapely shapely --force

!pip install --no-binary cartopy cartopy==0.17.0
# de quebra, instala o proplot
!pip install proplot

#### Segundo, o acesso ao API do CDS

Nada vai acontecer se você não instalar a chave de API do CDS e para isso você precisa criar uma conta no Copernicus e armazenar sua chave API exclusiva em um arquivo dot. Aqui estão as etapas para fazer isso:
1. Cria uma conta no Copernicus [aqui](https://cds.climate.copernicus.eu/)
2. Depois de criada, entre na sua conta no Copernicus ([aqui](https://cds.climate.copernicus.eu/user)), e anote o `UID`e o `API key`que aparece no final da página se você acabou de criar OU clique sobre o seu nome de usuário na página principal e note que ao final da página estará essa informação. Sugiro que você copie-cole em um notepad ou similar. 
3. Modifique o *code snippet* abaixo usando um editor de texto, modificando os valores de `<UID>` e `<API key>` com os que você gravou.

'url: https://cds.climate.copernicus.eu/api/v2'<br>
'key: \<UID\>:\<API key\>'<br>
'verify: 0'<br>


**Note** que o nome do arquivo deve ser `.cdsapirc` e ele deve ser criado na sua `$HOME` (independente se é no windows ou no linux). **(Nota especial para o COLAB!)**: O código abaixo serve para você garantir que seus dados de chave estarão na home da máquina virtual. Porém, **cuidado** se você for compartilhar esse notebook na rede pois ele contém a sua chave pessoal de acesso ao CDS. 

Você pode criar diretamente no terminal do linux ou via notepad no Windows. Como verificação de integridade (e sua sanidade mental), verifique o conteudo do arquivo `.cdsapirc` para garantir que tudo pareça correto. Um arquivo correto será semelhante a este aqui:

'url: https://cds.climate.copernicus.eu/api/v2'<br>
'key: 54321:a77a7a7a-7a77-7777-7777-aa77a7a77a7a'<br>
'verify: 0'<br>

(**Obviamente** não copie o conteudo do arquivo acima como seu porque não irá funcionar).
<br>

In [ ]:
# Se você usa o Jupyter, crie o arquivo .crsapirc "por fora" e pule essa etapa
# EXCLUSIVO COLAB ....criação da chave no home do COLAB
###### altere os dados para sua própria chave ######
!echo url: https://cds.climate.copernicus.eu/api/v2 > ~/.cdsapirc
!echo key: 00000:a111111z-11z1-11z0-abc1-1111a0aaaaz1 >> ~/.cdsapirc
!echo verify:0 >> ~/.cdsapirc
# e verifica se está tudo bem
!more ~/.cdsapirc

#### Segundo passo, instale o cliente CDS API

Algo que não foi mencionado antes mas é bastante salutar, é você usar um *ambiente virtual* ((veja aqui)[https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html]). Independente disso, a instalação do cliente `cdsapi`é feiot via `conda`ou `pip`:

`pip install cdsapi` (Principalmente no COLAB)
ou 
`conda install cdsapi`

Depois de instalado, certifique-se de que você pode importar e iniciar o cliente em um script Python sem erros, por exemplo crianto um pequeno programa ou executando a linha abaixo.
<br>

In [ ]:
# Exclusivo COLAB
# Instale agora a api do CDS para acesso.
!pip install cdsapi

In [ ]:
# e por fim, importe o seu cliente para baixar o ERA5
# Rode esse comando em qualquer ambiente, independente se é COLAB ou Jupyter
import cdsapi
cds = cdsapi.Client()

#### Agora ficou fácil! usando o cliente CDS API

O código abaixo mostra como solicitar temperaturas médias mensais em um nível de pressão especificado ao longo de um intervalo de datas. A saída é armazenada na memória como um `xarray` dataset e pode ser ativada ou desativada usando o `download_flag`:

In [ ]:
import warnings ## melhor ativar isso se você não quer encher sua tela com menssagens
warnings.filterwarnings("ignore")

In [ ]:
import cdsapi
import xarray as xr
from urllib.request import urlopen
# start the client
cds = cdsapi.Client()
# dataset you want to read
dataset = "reanalysis-era5-pressure-levels-monthly-means"
# flag to download data
download_flag = False
# api parameters 
params = {
    "format": "netcdf",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": '1000',
    'year':['2019','2020'],
    'month':['01','02','03'],
    "time": "00:00", 
    "grid": [1.0, 1.0],
    "area": [90, -180, -90, 180],
    }
# retrieves the path to the file
fl = cds.retrieve(dataset, params)
# download the file 
if download_flag:
    fl.download("./output.nc")
# load into memory
with urlopen(fl.location) as f:
    dset = xr.open_dataset(f.read())

In [ ]:
dset # para você dar uma olhadinha no que você baixou

In [ ]:
# note que você pode fazer experiências. Alguns campos tem 4 dimensões e outros 3. Adapte abaixo dependendo da sua necessidade;
#air = dset['t'][-1,1,:,:] -273.15
air = dset['t'][-1,:,:]-273.15

E agora podemos usar um código simples com o Proplot para plotar projeções com os campos globais de Temperatura do Ar (t), empregando também a função .contourf() do Matplotlib. Note que a temperatura está em Kelvin. Outro ponto relevante é que pode aparecer aqui uma dimensão `expver` (**1** ou **5**) que são as versões do ERA5 (`expver 1`) e ERA5-Land (`expver 5`) (veja explicação (aqui)[https://confluence.ecmwf.int/pages/viewpage.action?pageId=173385064]). Se necessário você precisará fazer uma pré-seleção antes de plotar. Modifique se necessário.

In [ ]:
import proplot as plot
fig, ax = plot.subplots(axheight=5, tight=True,
                         proj='pcarree',)

ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=10, lonlines=10,
          latlim=(12, -60), lonlim=(275, 330),
          title='Temperatura do Ar')

map1 = ax.contourf(dset['longitude'], dset['latitude'],
                   air,
                   cmap='coolwarm', levels=plot.arange(0, 30, 5),
                   extend='both')

fig.colorbar(map1, loc='r', label='Temperatura ($\degree$C)')
plot.show()

No lugar dos parâmetros de ano, mês, dia e hora, você pode, alternativamente, especificar uma lista de datas:

`'date':['2020-01-01 12:00', '2020-02-01 12:00', '2020-03-01 12:00'],`

ou você pode especificar uma data e hora como esta:

`'date':['2020-01-01', '2020-02-01', '2020-03-01'],`
`'time: '12:00',                                   `

Tanto faz você usar uma ou outra, o resultado é o mesmo na solicitação para a API. Essas solicitações são muito específicas e na maioria das vezes você pode solicitar um intervalo contínuo, como por exemplo `'date':'2020-01-01 12:00/2020-03-01 12:00'`.

Você ainda pode usar o pandas como um `date_range`, mas observe que a entrada para a API deve ser uma lista de *strings*. Aqui está um exemplo de uso de pandas:

`‘date’: list(pd.date_range(‘2020–01–01 12:00',’2020–03–01 12:00', freq=’M’).strftime(‘%Y-%m-%d %H:%M’)),` (**cautela com as aspas**).
<br>

#### Mais dicas úteis

**Mensagens de erro** <br>
Se nenhum dado está disponível no subconjunto solicitado e a solicitação não retornou dados, vai aparecer uma msg de erro.<br>
`no data is available within your requested subset. Request returned no data`
Isso geralmente indica que um valor de palavra-chave está fora do intervalo disponível. Por exemplo, apenas um conjunto específico de níveis de pressão pode ser solicitado. Além disso, o parâmetro de tempo deve ser para a hora mais próxima usando um relógio de 24 horas. Veja abaixo os erros comuns que vejo e como corrigi-los:

- O nível de pressão deve estar contido em: `[1, 2, 3, 5, 7, 10, 20, 30, 50, 70, range(100, 1000, 25)]`, onde a faixa (100,1000,25) indica 100 a 1000 em incrementos de 25. Se você receber um erro, verifique se a pressão que você está solicitando está contida nesta lista.
- O tempo deve corresponder à hora mais próxima em um relógio de 24 horas: `00:00, 01:00,…, 23:00`. Se você receber um erro, verifique se a hora é a hora mais próxima. Se você estiver solicitando médias mensais, certifique-se de que o parâmetro de tempo esteja definido como `‘00:00’`.

Existe um enorme [fórum](https://confluence.ecmwf.int/display/CUSF/forum) do CDS API e você pode perguntar suas dúvidas por lá. Não há uma documentação extensa do CDS API mas muitas informações podem ser obtidas [por aqui](https://cds.climate.copernicus.eu/api-how-to). Parte das palavras-chave permitidas e os nomes de variáveis variam dependendo do conjunto de dados. Os links a seguir são úteis:
- Os parâmetros de palavra-chave permitidos para cada conjunto de dados estão [neste link](https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+%28CDS%29+API+Keywords).
- As variáveis disponíveis estão listadas em [tabelas como esta](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation). Você fornece o que está listado no nome da variável na coluna CDS.
- Você também pode consultar o script de exemplo [aqui](https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+(CDS)+API+Keywords).

Outra opção é usar a interface da web para fazer uma seleção e ver “mostrar solicitação de API” na parte inferior. Por exemplo, use [este link](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means?tab=form) para escolher a "**média mensal dos níveis de pressão**". Outros conjuntos de dados podem ser encontrados [aqui](https://cds.climate.copernicus.eu/#!/search?text=ERA5&type=dataset). Clique no conjunto de dados que deseja acessar e, em seguida, na guia “Baixar dados” (ou `download data`) para fazer sua seleção. *O céu (ou o oceano) é o limite*.


#### Jupyterize seus scripts tornando a API mais fácil de usar

Quando você entender como funciona, você percebe que a API do CDS é extremamente muito poderosa. Compartilhe com seus colegas, "pythonize-se". Todas as informações aqui são de código aberto. Fique à vontade para copiá-la e modificá-la como desejar. Se usar o COLAB, melhor ainda porque dai qualquer pessoa mesmoooo poderá usar, em qualquer lugar.

<img src='https://www.python.org/static/community_logos/python-logo-master-v3-TM-flattened.png' width=150px></img>
<img src='https://cdn.iconscout.com/icon/free/png-256/love-chat-2125802-1789050.png' width=50px></img>
<img src='https://miro.medium.com/max/1400/1*7oukapIBInsovpHkQB3QZg.jpeg' width=150px></img>

